In [ ]:
%%javascript

IPython.OutputArea.auto_scroll_threshold = 10000

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
''')

In [ ]:
import os
import json
from ipywidgets import widgets
from ipywidgets import HBox, VBox
from ipywidgets import IntProgress
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

style = {'description_width': '100px'}
layout = {'width': '500px'}
button_descriptions  = {False: "Stop Tensorboard", True: "Launch Tensorboard"}

In [ ]:
class GuiWrapper:

    def __init__(self):
        self.style = {'description_width': '200px'}
        self.layout = {'width': '500px'}

    def generate_widgets(self, description, value=None, disabled=False):
        return widgets.Text(description=description, value=value, disabled=disabled,
                            style=self.style, layout=self.layout)

    def generate_button(self, description='Done', button_style='success'):
        return widgets.Button(description=description, layout=widgets.Layout(width='31%', height='30px'),
                              button_style=button_style, align_items='center')
    
    def generate_center_button(self, description='Done', button_style='success'):
        btn = widgets.Button(description=description, button_style='success',layout=widgets.Layout(width='50%', 
                                                                                                   height='30px'))
        box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='70%')
        box = widgets.HBox(children=[btn],layout=box_layout)
        display(box)
        return btn
    
    def get_drop_down_widgets(self,description, tuple_list, value):
        return widgets.Dropdown(options=tuple_list, value = value, description=description, layout=self.layout,
                                style=self.style,
                                disabled=False)
    

    def get_value(self, element_name_ref):
        return element_name_ref.value

In [ ]:
import copy
import json
import glob


class GetParam:
    def __init__(self):
        self.gui_wrapper = GuiWrapper()
        self.param_dict = {}
        self.dict_of_basic_ip = {}
        self.dict_for_general_info = {}
        self.dict_for_version = {}

        self.arg_name = None
        self.dummy_name = None
        self.arg_value = None
        self.arg_type = None
        self.arg_required = None
        self.section_name = None
        self.comment = None

        self.scope_1 = widgets.Output()
        self.scope_2 = widgets.Output()
        self.scope_3 = widgets.Output()

        self.show()

        self.done_button = None
        self.add_argument_button = None
        self.add_section_button = None
        self.generate_json_button = None
        self.list_of_dicts = []
        self.counter = 0
        self.list_for_create_json = []
        self.list_of_json_files = glob.glob('model_info.json')
        self.data_types_list = ['str', 'int', 'float', 'double', 'file', 'dir', 'flag']

    def generate_json_without_section(self, _):
        self.add_section_button.disabled = True
        if len(self.list_of_json_files) == 0:
            self.list_for_create_json = [{**self.dict_for_version}]
            self.list_for_create_json.append({**self.dict_for_general_info})
        self.list_for_create_json.append({**self.dict_of_basic_ip})
        self.dump_json()

    def dump_json(self):
        if len(self.list_of_json_files) > 0:
            models_info = json.load(open(self.list_of_json_files[0]))
            models_info.append(self.list_for_create_json[0])
            self.list_for_create_json = models_info
        with open("model_info.json", "w") as write_file:
            json.dump(self.list_for_create_json, write_file, indent=4)
        print('Json file is generated')

    def generate_json(self, _):
        self.generate_json_button.disabled = True
        self.add_section_button.disabled = True
        if len(self.list_of_json_files) == 0:
            self.list_for_create_json = [{**self.dict_for_version}]
            self.list_for_create_json.append({**self.dict_for_general_info})
        section_dict = {}
        for each_dict in self.list_of_dicts:
            section_dict = {**section_dict, **each_dict}
        self.dict_of_basic_ip.update({"sections": section_dict})
        self.list_for_create_json.append({**self.dict_of_basic_ip})
        with self.scope_2:
            clear_output()
            self.dump_json()

    def disable_buttons(self):
        self.add_argument_button.disabled = True
        self.done_button.disabled = True
        self.add_section_button.disabled = True

    def on_done_click(self, _):
        status = self.append_section()
        if status:
            self.disable_buttons()
            self.scope_3.clear_output()
            self.add_section_button = self.gui_wrapper.generate_button('Add section', 'warning')
            self.add_section_button.on_click(self.on_add_section_click)
            self.generate_json_button = self.gui_wrapper.generate_button("Generate Json",
                                                                         button_style='success')
            self.generate_json_button.on_click(self.generate_json)
            display(HBox([self.add_section_button, self.generate_json_button]))

    def append_section(self):
        if len(self.arg_name.value) > 0:
            self.append_comment_and_dict_obj()
            temp_dict = copy.deepcopy(self.param_dict)
            self.param_dict = {}
            section_name = self.section_name.value
            dict_of_section = {section_name: temp_dict}
            self.list_of_dicts.append(dict_of_section)
            self.section_name = None
            return True
        else:
            self.clear_scope_3()
            return False

    def clear_scope_3(self):
        with self.scope_3:
            clear_output()
            print('Please provide Argument name')

    def append_comment_and_dict_obj(self):

        if self.arg_required.value == 'true':
            field_req = True
        else:
            field_req = False

        if self.arg_value.value is None or len(self.arg_value.value) == 0 or \
                self.dummy_name is None or len(self.dummy_name.value) == 0:
            if self.arg_value.value is None or len(self.arg_value.value) == 0:
                if self.dummy_name is None or len(self.dummy_name.value) == 0:
                    if self.arg_type.value == 'dir':
                        dict_obj = {self.arg_name.value: {'type': self.arg_type.value,
                                                          'required': field_req}}
                    else:
                        dict_obj = {self.arg_name.value: {'default': "",
                                                          'type': self.arg_type.value,
                                                          'required': field_req}}
                else:
                    if self.arg_type.value == 'dir':
                        dict_obj = {self.arg_name.value: {'name': self.dummy_name.value,
                                                          'type': self.arg_type.value,
                                                          'required': field_req}}
                    else:
                        dict_obj = {self.arg_name.value: {'name': self.dummy_name.value,
                                                          'default': "",
                                                          'type': self.arg_type.value,
                                                          'required': field_req}}
            else:
                if self.arg_type.value == 'dir':
                    dict_obj = {self.arg_name.value: {'type': self.arg_type.value,
                                                      'required': field_req}}
                else:
                    dict_obj = {self.arg_name.value: {'default': self.arg_value.value,
                                                      'type': self.arg_type.value,
                                                      'required': field_req}}
        else:
            if self.arg_type.value == 'dir':
                dict_obj = {self.arg_name.value: {'name': self.dummy_name.value,
                                                  'type': self.arg_type.value,
                                                  'required': field_req}}
            else:
                dict_obj = {self.arg_name.value: {'name': self.dummy_name.value,
                                                  'default': self.arg_value.value,
                                                  'type': self.arg_type.value,
                                                  'required': field_req}}

        self.param_dict.update(dict_obj)

    def add_to_param_dict(self):
        if len(self.arg_name.value) > 0:
            self.append_comment_and_dict_obj()
            return True
        else:
            self.clear_scope_3()
            return False

    def rest_variables(self):
        self.arg_name = None
        self.dummy_name = None
        self.arg_value = None
        self.arg_type = None
        self.arg_required = None

    def show(self):
        display(VBox([self.scope_1, self.scope_2, self.scope_3]))

    def on_add_argument_click(self, _):
        status = self.add_to_param_dict()
        if status:
            self.scope_3.clear_output()
            self.disable_buttons()
            self.add_arg_utils()

    def add_arg_utils(self):
        self.rest_variables()
        self.set_text_area_for_arg()
        self.add_or_done_functionality()

    def set_text_area_for_arg(self, default_name='', default_value='', default_arg_type=None,
                              default_dummy_name=''):
        self.arg_name = self.gui_wrapper.generate_widgets('Arg name', default_name)
        self.dummy_name = self.gui_wrapper.generate_widgets('Name', default_dummy_name)
        self.arg_value = self.gui_wrapper.generate_widgets('Default', default_value)

        if default_arg_type is None:
            self.arg_type = self.gui_wrapper.get_drop_down_widgets('Data type', self.data_types_list,
                                                                   self.data_types_list[0])
        else:
            self.arg_type = self.gui_wrapper.get_drop_down_widgets('Data type', self.data_types_list,
                                                                   default_arg_type)
        self.arg_required = self.gui_wrapper.get_drop_down_widgets('Required', ['true', 'false'], 'true')
        display(widgets.VBox([self.arg_name, self.dummy_name, self.arg_value, self.arg_type,
                              self.arg_required]))

    def add_or_done_functionality(self):
        print('To add more arguments click on Add argument or to proceed click on Done')
        self.done_button = self.gui_wrapper.generate_button('Done', 'success')
        self.add_argument_button = self.gui_wrapper.generate_button('Add argument', 'warning')
        self.add_argument_button.on_click(self.on_add_argument_click)
        self.done_button.on_click(self.on_done_click)
        display(HBox([self.add_argument_button, self.done_button]))

    def set_param_info_and_generate_json(self):
        self.add_section()

    def on_add_section_click(self, _):
        with self.scope_1:
            self.scope_3.clear_output()
            clear_output()
            self.on_add_section_click_first_time(_)

    def on_add_section_click_first_time(self, _):
        with self.scope_1:
            clear_output()
            self.counter += 1
            temp_section = 'section_' + str(self.counter)
            self.section_name = self.gui_wrapper.generate_widgets('Section name', temp_section)
            self.comment = self.gui_wrapper.generate_widgets('Give the comment: ', ' section comment')
            self.add_section_button.disabled = True
            display(HBox([self.section_name]))
            display(widgets.VBox([self.comment]))
            self.set_text_area_for_default_arg()

    def set_text_area_for_default_arg(self, default_name='', default_type='', default_value='', default_req='true'):
        self.arg_name = self.gui_wrapper.get_drop_down_widgets('Command arg name', ['command'], 'command')
        self.arg_required = self.gui_wrapper.get_drop_down_widgets('run_type', ['python3', 'tensorboard'],
                                                                   'python3')
        display(widgets.VBox([self.arg_name, self.arg_required]))
        proceed_btn = self.gui_wrapper.generate_center_button('Proceed', 'success')
        proceed_btn.on_click(self.on_proceed_click)

    def on_proceed_click(self, btn):
        self.add_default_param_to_dict()
        btn.disabled = True

    def append_comment_and_default_dict_obj(self):
        self.param_dict.update({'_comment': self.comment.value})
        dict_obj = {self.arg_name.value: {'run_type': self.arg_required.value}}
        self.param_dict.update(dict_obj)

    def add_default_param_to_dict(self):
        self.append_comment_and_default_dict_obj()
        if self.arg_required.value == 'python3':
            self.set_text_area_for_arg(default_name='execution_script', default_value='src/train.py',
                                       default_arg_type='file', default_dummy_name='Execution Script')
        else:
            self.set_text_area_for_arg(default_name='logdir', default_arg_type='dir',
                                       default_value='training.train_dir', default_dummy_name='Eventfile Path')
        self.add_or_done_functionality()

    def add_section(self):
        with self.scope_1:
            clear_output()
            print('If you want to add section click on Add section, else click on Generate JSON')
            json_button = self.gui_wrapper.generate_button('Generate JSON', 'success')
            self.add_section_button = self.gui_wrapper.generate_button('Add section', 'warning')
            self.add_section_button.on_click(self.on_add_section_click_first_time)
            json_button.on_click(self.generate_json_without_section)
            display(HBox([self.add_section_button, json_button]))


In [ ]:
class GenerateJson:

    def __init__(self):
        print('************************* Please provide the model basic info ***********************')
        self.gui_inst = GuiWrapper()
        self.model_name = self.gui_inst.generate_widgets('Model name', 'Human Count 2.1')
        self.git_url = self.gui_inst.generate_widgets('Git url',
                                                      'https://github.com/LatticeSemi/Human-Count.git -b 2.1')
        self.target_device = self.gui_inst.generate_widgets('Target device', 'ECP5')
        self.target_application = self.gui_inst.generate_widgets('Target application', 'Object Detection')
        self.target_class = self.gui_inst.get_drop_down_widgets('Target class', ['CNN', 'BNN'], 'CNN')

        self.input_dim = self.gui_inst.generate_widgets('Input dim', '224x224')
        self.target_network = self.gui_inst.generate_widgets('Target network', 'SqueezeDet')
        self.target_framework = self.gui_inst.generate_widgets('Target framework', 'TensorFlow')
        self.sensAI_version = self.gui_inst.generate_widgets('sensAI version',
                                                             "['V2.0.0', 'V2.1.0-Beta', 'V3.0']")
        self.other_param = self.gui_inst.generate_widgets('other_param',
                                                             "")

        self.dict_of_basic_ip = {}
        display(widgets.VBox(
            [self.model_name, self.git_url, self.target_device, self.target_application, self.target_class,
             self.input_dim, self.target_network, self.target_framework, self.sensAI_version, self.other_param]))
        done_button = self.gui_inst.generate_center_button('Next', 'success')
        done_button.on_click(self.set_basic_model_and_parameter_info)

        # display(widgets.VBox([done_button]))
        self.scope_1 = widgets.Output()
        self.is_basic_config_done = False
        self.general_info_dict ={
		"model_name": "default_example. This should be unique for each model entry",
		"git_url": "This should be git url of the model. To clone a particular branch specify in url itself.",
		"model_spec": {
			"target_device": "ECP5, Ultra Plus and more.",
			"target_application": "This will be the type of application this model targets e.g. Object Detection, Face Detection, etc",
			"target_class": "BNN or CNN.",
			"input_dim": "This will be input dimension for images to feed the network e.g. 224x224, 64x64, etc.",
			"target_network": "This can network architecture type e.g. YOLO, squeezeDet, etc.",
			"target_framework": "This will be the framework used e.g TensorFlow or Keras.",
			"sensAI_version": "Target sensAI version. Ex : ['V2.0.0', 'V2.1.0-Beta', 'V3.0']",
            "other_param": "This will be extra param to differentiate same Model but different version"
		},
		"sections": {
			"_comment": "Here User can define multiple sections like training, freeze, tensorboard, evaluation and many more.",
			"example_section": {
				"_comment": "Section name should unique for one model. Section contains information that how to run the section. Section has mainly three type of parameters, command, execution_script and other_parameters.",
				"command": {
					"run_type": "This parameter determines how to run the given file. Ex: python3, shell, tensorboard etc."
				},
				"execution_script": {
					"default": "Path to file from model's root directory",
					"type": "file",
					"required": 'true'
				},
				"other_parameters": {
					"default": "This contains default value for perticular parameter.",
					"type": "This contains type of parameter. Like file, dir, int, str, flag, float",
					"required": "true or false based on parameter is required or not."
				}
			}
		}
	}
        self.dict_of_version = {"version": 0.2}

    def set_basic_model_and_parameter_info(self, button_inst):
        with self.scope_1:
            clear_output()
            self.dict_of_basic_ip = {'model_name': self.model_name.value,
                                     'git_url': self.git_url.value,
                                     "model_spec": {
                                         "target_device": self.target_device.value,
                                         "target_application": self.target_application.value,
                                         "target_class": self.target_class.value,
                                         "input_dim": self.input_dim.value,
                                         "target_network": self.target_network.value,
                                         "target_framework": self.target_framework.value,
                                         "sensAI_version": self.sensAI_version.value,
                                         "other_param": self.other_param.value
                                     }
                                     }
            self.is_basic_config_done = True
            parameter_inst = GetParam()
            parameter_inst.dict_of_basic_ip = self.dict_of_basic_ip
            parameter_inst.dict_for_general_info = self.general_info_dict
            parameter_inst.dict_for_version = self.dict_of_version
            parameter_inst.set_param_info_and_generate_json()
            button_inst.disabled = True

    def show(self):
        display(VBox([self.scope_1]))


In [ ]:
inst = GenerateJson()
inst.show()

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code.">
</form>''')